In [1]:
#Author: Anshul Kapoor
#Code Purpose: Make Single JSON for all EGamma SFs

#All the files for SFs
files=[
    "EGdata/egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root",
    "EGdata/egammaEffi.txt_Ele_Loose_postVFP_EGM2D.root",
    "EGdata/egammaEffi.txt_Ele_Medium_postVFP_EGM2D.root",
    "EGdata/egammaEffi.txt_Ele_Tight_postVFP_EGM2D.root",
]

#Names that you want for SFs for files above (same order please)
names=["Veto","Loose","Medium","Tight"]
#Names that you want for errors for files above (same order please)
nameserrs=["Veto_err","Loose_err","Medium_err","Tight_err"]
nameJSON="egcorrs_withErrors.json" # Name of final JSON

sfhist="EGamma_SF2D"

In [2]:
from ROOT import TFile, TH1F, TCanvas, TString

#Function to extract SFs from EGamma standard root files
def getSFs(fn="filename"):
    tf = TFile(fn)
    fo = tf.Get(sfhist)
    Xbins=fo.GetNbinsX()
    Ybins=fo.GetNbinsY()
    X=[None]*(Xbins+1)
    Y=[None]*(Ybins+1)
    values=[]
    errors=[]
    for i in range(1,Xbins + 1):
        X[i-1]=(fo.GetXaxis().GetBinLowEdge(i))
    X[Xbins]=fo.GetXaxis().GetBinUpEdge(Xbins)
    for j in range(1,Ybins + 1):
        Y[j-1]=(fo.GetYaxis().GetBinLowEdge(j))
    Y[Ybins]=fo.GetYaxis().GetBinUpEdge(Ybins)
    for i in range(1,Xbins + 1):
        for j in range(1,Ybins + 1):
            values.append(fo.GetBinContent(i,j))
            errors.append(fo.GetBinError(i,j))
    return X,Y,values,errors


#Use correctionlib to prepare jsons for all wps
from correctionlib.schemav1 import Correction
corrs=[]
for name,name_err,file in zip(names,nameserrs,files):
    X,Y,values,errors=getSFs(file) #Using getSFs function defined above
    
    corr = Correction.parse_obj(
    {
        "version": 1,
        "name": name,
        "inputs": [
            {"name": "pt", "type": "real"},
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
          
        ],
        "output": {"name": "weight", "type": "real"},
        "data": {
            "nodetype": "multibinning",
            "edges": [
                Y,
                X,
            ],
            "content": values,
        },
    })
    corrs.append(corr)
    
    correrr = Correction.parse_obj(
    {
        "version": 1,
        "name": name_err,
        "inputs": [
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": {
            "nodetype": "multibinning",
            "edges": [
                Y,
                X,
            ],
            "content": errors,
        },
    })
    corrs.append(correrr)

#Save JSON
from correctionlib.schemav1 import CorrectionSet
import gzip

cset = CorrectionSet.parse_obj({
    "schema_version": 1,
    "corrections": corrs,
})

with open(nameJSON, "w") as fout:
    fout.write(cset.json(exclude_unset=True, indent=4))

Welcome to JupyROOT 6.22/06


In [3]:
#Evaluator Example

import libcorrection

cset = libcorrection.CorrectionSet(nameJSON)

cset["Veto"]

In [4]:
valsf= cset["Medium"].evaluate(34.0, 1.1)
valsferr= cset["Medium_err"].evaluate(34.0, 1.1)

print("SF is:"+str(valsf))
print("error is:"+str(valsferr))

SF is:0.9831932783126832
error is:0.014436147986067594


In [5]:
#Testing a few things 
X,Y,values,errors=getSFs("EGdata/egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root")

In [6]:
X

[-2.5, -2.0, -1.566, -1.444, -0.8, 0.0, 0.8, 1.444, 1.566, 2.0, 2.5]

In [7]:
Y

[10.0, 20.0, 35.0, 50.0, 100.0, 500.0]

In [9]:

len(values)

50

In [10]:
errors

[0.010571704486386103,
 0.015161713327338984,
 0.003326720994586664,
 0.003337786209726554,
 0.017076009800036644,
 0.015908888635886853,
 0.010904415622441719,
 0.0021097046413502706,
 0.0032410168259563703,
 0.007629628269684305,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.022211934730639107,
 0.013436816646698298,
 0.00222127784665934,
 0.005951910110379365,
 0.007558807705375244,
 0.0185955853111187,
 0.013050683805852583,
 0.0017985989694381537,
 0.002783030908079795,
 0.0052315854352529105,
 0.0185955853111187,
 0.013050683805852583,
 0.0017985989694381537,
 0.002783030908079795,
 0.0052315854352529105,
 0.022060619448594467,
 0.013436816646698298,
 0.00222127784665934,
 0.005951910110379365,
 0.007558807705375244,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.015908888635886853,
 0.010904415622441719,
 0.0021097046413502706,
 0.0032410168259563703,
 0.007460719371943233,
 0.011089503460718486,
 0.015161713327338984,
 0.003326720994586664,
 0.003337786209726554,
 0.017138097060249706]

In [47]:
import uproot4
sf = uproot4.open("EGdata/egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root:EGamma_SF2D").to_boost()

In [48]:
import numpy
list(sf.view().value.flatten())

[1.0217949151992798,
 0.993127167224884,
 0.9924487471580505,
 0.9871657490730286,
 1.0181623697280884,
 1.015222430229187,
 0.9923245906829834,
 0.996835470199585,
 1.0010526180267334,
 1.0180851221084595,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0050188302993774,
 0.9875706434249878,
 0.9895397424697876,
 0.9885655045509338,
 1.0073375701904297,
 0.9701120853424072,
 0.9832962155342102,
 0.9885773658752441,
 0.9886363744735718,
 1.0104166269302368,
 0.9763681888580322,
 0.9866518378257751,
 0.9906542301177979,
 0.9917269945144653,
 1.0031217336654663,
 1.0138190984725952,
 0.9864253401756287,
 0.9916142821311951,
 0.9895941615104675,
 1.0073375701904297,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0285035371780396,
 0.9812362194061279,
 0.9936708807945251,
 0.993697464466095,
 0.9896373152732849,
 1.0149999856948853,
 0.9908779859542847,
 0.9892008900642395,
 0.9829059839248657,
 1.0063626766204834]

In [49]:
tf = TFile("EGdata/egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root")
fo = tf.Get("EGamma_SF2D")
Xbins=fo.GetNbinsX()
Ybins=fo.GetNbinsY()
X=[None]*(Xbins+1)
Y=[None]*(Ybins+1)
values=[]
errors=[]
for i in range(1,Xbins + 1):
    X[i-1]=(fo.GetXaxis().GetBinLowEdge(i))
X[Xbins]=fo.GetXaxis().GetBinUpEdge(Xbins)
for j in range(1,Ybins + 1):
    Y[j-1]=(fo.GetYaxis().GetBinLowEdge(j))
Y[Ybins]=fo.GetYaxis().GetBinUpEdge(Ybins)

In [50]:
for i in range(1,Xbins + 1):
    for j in range(1,Ybins + 1):
        print(str(i)+', '+str(j))
        values.append(fo.GetBinContent(i,j))
        errors.append(fo.GetBinError(i,j))

1, 1
1, 2
1, 3
1, 4
1, 5
2, 1
2, 2
2, 3
2, 4
2, 5
3, 1
3, 2
3, 3
3, 4
3, 5
4, 1
4, 2
4, 3
4, 4
4, 5
5, 1
5, 2
5, 3
5, 4
5, 5
6, 1
6, 2
6, 3
6, 4
6, 5
7, 1
7, 2
7, 3
7, 4
7, 5
8, 1
8, 2
8, 3
8, 4
8, 5
9, 1
9, 2
9, 3
9, 4
9, 5
10, 1
10, 2
10, 3
10, 4
10, 5


In [51]:
values

[1.0217949151992798,
 0.993127167224884,
 0.9924487471580505,
 0.9871657490730286,
 1.0181623697280884,
 1.015222430229187,
 0.9923245906829834,
 0.996835470199585,
 1.0010526180267334,
 1.0180851221084595,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0050188302993774,
 0.9875706434249878,
 0.9895397424697876,
 0.9885655045509338,
 1.0073375701904297,
 0.9701120853424072,
 0.9832962155342102,
 0.9885773658752441,
 0.9886363744735718,
 1.0104166269302368,
 0.9763681888580322,
 0.9866518378257751,
 0.9906542301177979,
 0.9917269945144653,
 1.0031217336654663,
 1.0138190984725952,
 0.9864253401756287,
 0.9916142821311951,
 0.9895941615104675,
 1.0073375701904297,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0285035371780396,
 0.9812362194061279,
 0.9936708807945251,
 0.993697464466095,
 0.9896373152732849,
 1.0149999856948853,
 0.9908779859542847,
 0.9892008900642395,
 0.9829059839248657,
 1.0063626766204834]

In [54]:
numpy.array(list(sf.view().value.flatten())) - numpy.array(values)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])